<a href="https://colab.research.google.com/github/Zafar-Lab/scDREAMER/blob/main/2109_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" # difference between gpu '0' and '1'

"""
# for reproducibility with server...

os.environ["PYTHONHASHSEED"] = "0"
os.environ["TF_CUDNN_DETERMINISTIC"] = "1" # setting "true"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
"""

import numpy as np
import tensorflow as tf2
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import scipy.io
from sklearn.decomposition import PCA
import pdb

import pandas as pd
import scanpy as sc


import scipy.sparse
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy import stats 
from scipy import * 
import datetime 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

os.getpid()


2023-09-25 15:19:33.890043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 15:19:34.558038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


2529549

In [2]:

np.random.seed(0)
tf.set_random_seed(0)
random.seed(0)
tf2.random.set_seed(0)
tf2.keras.utils.set_random_seed(0)


In [3]:
import warnings
warnings.filterwarnings('ignore')
"""
!pip install -U scipy==1.5.0
!pip install scanpy==1.8.2
!pip install -U scikit-learn
"""

'\n!pip install -U scipy==1.5.0\n!pip install scanpy==1.8.2\n!pip install -U scikit-learn\n'

Building model

In [4]:


name = "Pancreas"
name = "Human_Retina"
name = "Lung"
name = "Healthy_Heart"

name = "Human_Mouse"
name = "Lung"
name = "Immune_Human"


In [5]:


data_path = {
"Immune_Human" : "/home/ajita/Documents/data_integration/Immune/Immune_ALL_human.h5ad",
"Pancreas" : "/home/ajita/Documents/data_integration/Pancreas/Pancreas.h5ad",
"Human_Mouse" : "/home/ajita/Documents/data_integration/hum_mou/hcl_mca_merged.h5ad",
"Human_Retina" : "/home/ajita/Documents/data_integration/Human_Retina/Human_Retina.h5ad",
"Lung" : "/home/ajita/Documents/data_integration/Lung/Lung_atlas_public.h5ad",
"Healthy_Heart" : "/home/ajita/Documents/data_integration/Healthy_Heart/Healthy_human_heart_adata.h5ad" 
}


batch_key_dict = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch",
                 "Healthy_Heart" : "sampleID"
                 
                }
cell_type_key_dict = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',  
                 'Pancreas' : 'celltype', 
                 'Human_Mouse' : "celltype", 
                 "Human_Retina":"Subcluster",
                "Healthy_Heart" : "celltype"
                    }    

shuffle_type_dict = {'Immune_Human' : 1, 'Lung' : 2, 'Pancreas' : 1, 'Human_Mouse' : 1, 
                 "Human_Retina":2,"Healthy_Heart" : 1
                    }
learning_rate = {
                 'Immune_Human' : {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Lung' : {"lr_ae" : 0.0002 , "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Pancreas' : {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Human_Retina': {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Human_Mouse' : {"lr_ae" : 0.0001, "lr_dis": 0.00001, "lr_bc": 0.0007},
                 "Healthy_Heart" : {"lr_ae" : 0.0001, "lr_dis": 0.00001, "lr_bc": 0.0007}
}


# big datasets: 20 dimensions
latent_dim = {
                'Immune_Human' : 10,
                 'Lung' : 10,
                 'Pancreas' : 10,
                 'Human_Mouse' : 20,
                 'Human_Retina': 10,
                 "Healthy_Heart" : 20
    
            }
BS = {
    'Immune_Human' : 128,
     'Lung' : 128,
     'Pancreas' : 128,
     'Human_Mouse' : 128,
     'Human_Retina': 128,
     "Healthy_Heart" : 128
}

epoch_dict = {
    'Immune_Human' : 150,
     'Lung' : 250,
     'Pancreas' : 300,
     'Human_Mouse' : 200,
     'Human_Retina': 300,
     "Healthy_Heart" : 300
}

"""
Below variable takes care of sparse matrix
If 1: pd.DataFrame.sparse.from_spmatrix(Ann.X) is used
If 0: pd.DataFrame(Ann.X)

By default 0, unless scarcity not handled by basis pandas operation
"""

sparseIP= {
        "Healthy_Heart" : 1,
    'Immune_Human' : 0, 
     'Lung' : 1,
     'Pancreas' : 0,
     'Human_Mouse' : 1,
     "Human_Retina": 0
}


In [ ]:
    
# add avg of losses,

import src.model as model


run_config = tf.ConfigProto()

run_config.gpu_options.per_process_gpu_memory_fraction = 0.333
run_config.gpu_options.allow_growth = True

with tf.Session(config = run_config) as sess:

    dreamer = model.scDREAMER(
        sess,
        epoch = epoch_dict[name],
        dataset_name = data_path[name],
        batch = batch_key_dict[name],
        cell_type = cell_type_key_dict[name],
        name = name,
        lr_ae = learning_rate[name]['lr_ae'],
        lr_dis = learning_rate[name]['lr_dis'],
        lr_bc = learning_rate[name]['lr_bc'],
        batch_size = BS[name],
        z_dim = latent_dim[name],
        shuffle_type = shuffle_type_dict[name],
        sparseIP = sparseIP[name]
        )

    dreamer.train_cluster()



2023-09-25 15:19:37.366238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5367 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:d8:00.0, compute capability: 7.5


came in load_gene
updated hvg
here [2 4 1 ... 1 2 2]
Data set to work on:
[[0.6259505  0.7146579  0.         ... 0.         0.54856277 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.48574623 0.         ... 0.         0.49434054 0.        ]
 [0.         0.54886496 0.         ... 0.         0.41306633 0.        ]
 [0.         0.         0.         ... 0.         0.35608888 0.        ]]
(33506, 2000)
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(33506, 10)
encoder input shape  Tensor("concat:0", shape=(?, 2010), dtype=float32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.cast` instead.
decoder input shape  Tensor("concat_2:0", shape=(?, 20), dtype=

2023-09-25 15:19:52.690544: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Epoch : [0] ,  a_loss = 437.2694, d_loss: -0.2112 ,  db_loss: 1.7259
Epoch : [1] ,  a_loss = 402.7577, d_loss: -0.1035 ,  db_loss: 1.4702
Epoch : [2] ,  a_loss = 397.1234, d_loss: -0.1353 ,  db_loss: 1.4212
Epoch : [3] ,  a_loss = 391.1337, d_loss: -0.0417 ,  db_loss: 1.6001
Epoch : [4] ,  a_loss = 404.3828, d_loss: -0.0457 ,  db_loss: 1.7128
Epoch : [5] ,  a_loss = 388.6531, d_loss: -0.0142 ,  db_loss: 1.8067
Epoch : [6] ,  a_loss = 397.0950, d_loss: -0.0100 ,  db_loss: 1.4732
Epoch : [7] ,  a_loss = 396.6759, d_loss: -0.0519 ,  db_loss: 1.5876
Epoch : [8] ,  a_loss = 373.8354, d_loss: -0.0228 ,  db_loss: 1.6606
Epoch : [9] ,  a_loss = 377.7234, d_loss: -0.0273 ,  db_loss: 1.5762
Epoch : [10] ,  a_loss = 379.1226, d_loss: -0.0801 ,  db_loss: 1.6927
Epoch : [11] ,  a_loss = 385.0262, d_loss: -0.0497 ,  db_loss: 1.5672
Epoch : [12] ,  a_loss = 373.5075, d_loss: -0.0368 ,  db_loss: 1.6153
Epoch : [13] ,  a_loss = 371.1310, d_loss: -0.0724 ,  db_loss: 1.7159
Epoch : [14] ,  a_loss = 374.8

Epoch : [117] ,  a_loss = 322.8941, d_loss: -0.1851 ,  db_loss: 1.8220
Epoch : [118] ,  a_loss = 326.2980, d_loss: -0.0986 ,  db_loss: 1.8580
Epoch : [119] ,  a_loss = 330.3260, d_loss: -0.1225 ,  db_loss: 1.7615
Epoch : [120] ,  a_loss = 334.3076, d_loss: -0.1062 ,  db_loss: 1.6953
Epoch : [121] ,  a_loss = 332.1434, d_loss: -0.0758 ,  db_loss: 1.8166
Epoch : [122] ,  a_loss = 328.4632, d_loss: -0.1169 ,  db_loss: 1.7177
Epoch : [123] ,  a_loss = 334.0211, d_loss: -0.1042 ,  db_loss: 1.6956
Epoch : [124] ,  a_loss = 334.9138, d_loss: -0.1271 ,  db_loss: 1.6945
Epoch : [125] ,  a_loss = 321.2042, d_loss: -0.1026 ,  db_loss: 1.8950
Epoch : [126] ,  a_loss = 329.8711, d_loss: -0.0917 ,  db_loss: 1.7618
Epoch : [127] ,  a_loss = 329.7969, d_loss: -0.1144 ,  db_loss: 1.6680
Epoch : [128] ,  a_loss = 324.5728, d_loss: -0.1119 ,  db_loss: 1.8735
Epoch : [129] ,  a_loss = 334.6926, d_loss: -0.1339 ,  db_loss: 1.7244
Epoch : [130] ,  a_loss = 325.8275, d_loss: -0.0771 ,  db_loss: 1.8136
Epoch 